In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import numpy as np
from keras import backend as K
from experiment_constant import *
import pickle
from experiment_mnist import *
import pickle

/home/sergei/miniconda3/envs/ML/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dta = MNISTExperiment([10], 0.1, 10, epochs = 0, activation = 'sigmoid', reg_type = 'delta')
x = np.vstack((dta.x_train, dta.x_test))
y = np.vstack((dta.y_train, dta.y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
Total params: 7,960
Trainable params: 7,960
Non-trainable params: 0
_________________________________________________________________


In [3]:
L = 3
KLips = 2

In [4]:
#P = 0.3
#P = [0.3] + [0] * (L - 1)
#experiment = MNISTExperiment([11, 12, 13], P, KLips, epochs = 100, activation = 'relu', reg_type = 0,
#                             reg_coeff = 0, do_print = True)
#for a, b in zip(experiment.model_no_dropout.get_weights(), experiment.model.get_weights()):
#    assert np.allclose(a, b)
#weights = experiment.model_no_dropout.get_weights()
#W = weights[0::2]
#B = weights[1::2]
#pickle.dump([W, B], open('mnistnet100percenttightness.pkl', 'wb'))

In [5]:
[W, B] = pickle.load(open('mnistnet100percenttightness.pkl', 'rb'))
experiment = ConstantExperiment([28 ** 2, 11, 12, 13, 10], [0.3, 0, 0], 2, W, B, activation = 'relu')

In [6]:
experiment.update_C(x[:1000])
inp = x[5]#np.random.randn(28 ** 2)
errors = experiment.get_error(inp, repetitions = 1000)
mean_exp = np.mean(np.abs(errors))

In [7]:
experiment.use_Carr = True
mean_bound, std_bound = experiment.get_mean_std_error()
#print(mean_bound)
arr = mean_exp / mean_bound
print(np.mean(arr), np.min(arr), np.max(arr), np.median(arr))
#print(np.min(mean_bound))

(12, 11) 11 (12,)
(13, 12) 12 (13,)
(10, 13) 13 (10,)
0.02136026889703727 0.015037469575008632 0.03698968364472959 0.019461267725511083


In [8]:
experiment.use_Carr = False
mean_bound, std_bound = experiment.get_mean_std_error()
#print(mean_bound)
print(mean_exp / mean_bound)

0.005918942358473631
